# Fine-tuning a model on a question-answering task

Dieses Notebook basiert auf dem Beispiel von folgenden Notebook https://github.com/huggingface/notebooks/blob/main/examples/question_answering.ipynb und wurde auf unsere Daten angepasst.

In [ ]:
import pandas as pd

In [ ]:
from datasets import load_dataset, load_metric, Dataset

dataset = load_dataset('csv', data_files='test_data.csv')  
dataset1 = pd.read_csv('test_data.csv')

Using custom data configuration default-85f746ff15b73814


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-85f746ff15b73814/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Hash', 'question', 'answers.text', 'context', 'answers.answer_start'],
        num_rows: 100
    })
})

In [ ]:
datasets = Dataset.from_pandas(dataset1).train_test_split(test_size=0.2)

train_dataset = datasets["train"]
val_dataset = datasets["test"]


In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['Hash', 'question', 'answers.text', 'context', 'answers.answer_start'],
        num_rows: 80
    })
    test: Dataset({
        features: ['Hash', 'question', 'answers.text', 'context', 'answers.answer_start'],
        num_rows: 20
    })
})

In [ ]:
datasets['train'][0]

{'Hash': 'D3aVvm+J4AJVDebLuIAQDg==',
 'answers.answer_start': 3511,
 'answers.text': 'die aktuellen Werte von Variablen in die Einträge in Aktion / Name, Parameter und Erwartung / Wert übernommen werden.',
 'context': 'Diese Testschrittvorlage dient der Gruppierung zusammenhängender Testschritte. Lange Sequenzen lassen sich damit deutlich übersichtlicher gestalten. Im Testfalleditor können die im Block enthaltenen Schritte über das „+“-Zeichen neben dem Block eingeblendet oder mittels „-“ ausgeblendet werden. Im Testreport werden alle Testschritte innerhalb des Blocks eingerückt dargestellt. Die Bewertung ergibt sich durch Akkumulation der Einzelbewertungen der enthaltenen Testschritte. Der Block-Testschritt bietet die Möglichkeit zum Abbruch des Testfalls, wenn die Ausführung eines Testschritts innerhalb des Blocks fehlschlägt. Um das Abbruch-Verhalten eines Blocks zu definieren, muss auf den Pfeil neben „Bedingter Testabbruch“ geklickt werden. Als Abbruchbedingung wird die Menge von 

# Preprocessing the training data

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained("deutsche-telekom/bert-multi-english-german-squad2")


Downloading:   0%|          | 0.00/264 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/817 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.


for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 512:
        break
example = datasets["train"][i]

Token indices sequence length is longer than the specified maximum sequence length for this model (1046 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
len(tokenizer(example["question"], example["context"])["input_ids"])


1046

In [ ]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])


512

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)


In [ ]:
[len(x) for x in tokenized_example["input_ids"]]

[512, 512, 312]

In [ ]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))


[CLS] Kann ich die Werte von Variablen in Einträge übernehmen? [SEP] Diese Testschrittvorlage dient der Gruppierung zusammenhängender Testschritte. Lange Sequenzen lassen sich damit deutlich übersichtlicher gestalten. Im Testfalleditor können die im Block enthaltenen Schritte über das „ + [UNK] - Zeichen neben dem Block eingeblendet oder mittels „ - [UNK] ausgeblendet werden. Im Testreport werden alle Testschritte innerhalb des Blocks eingerückt dargestellt. Die Bewertung ergibt sich durch Akkumulation der Einzelbewertungen der enthaltenen Testschritte. Der Block - Testschritt bietet die Möglichkeit zum Abbruch des Testfalls, wenn die Ausführung eines Testschritts innerhalb des Blocks fehlschlägt. Um das Abbruch - Verhalten eines Blocks zu definieren, muss auf den Pfeil neben „ Bedingter Testabbruch [UNK] geklickt werden. Als Abbruchbedingung wird die Menge von Bewertungen angegeben, die als Fehlschlagen zu interpretieren sind. Die im Block enthaltenen Testschritte werden so lange ausg

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 3), (3, 4), (5, 8), (9, 12), (13, 18), (19, 22), (23, 26), (26, 31), (31, 32), (33, 35), (36, 39), (39, 44), (45, 55), (55, 56), (0, 0), (0, 5), (6, 11), (11, 13), (13, 17), (17, 20), (20, 24), (25, 30), (31, 34), (35, 40), (40, 46), (47, 55), (55, 58), (58, 64), (65, 70), (70, 72), (72, 76), (76, 77), (77, 78), (79, 84), (85, 87), (87, 92), (92, 94), (95, 101), (102, 106), (107, 112), (113, 121), (122, 126), (126, 135), (135, 137), (138, 140), (140, 144), (144, 147), (147, 148), (149, 151), (152, 156), (156, 160), (160, 165), (165, 166), (167, 173), (174, 177), (178, 180), (181, 186), (187, 196), (196, 198), (199, 206), (206, 207), (208, 212), (213, 216), (217, 218), (218, 219), (219, 220), (220, 221), (221, 228), (229, 234), (235, 238), (239, 244), (245, 248), (248, 250), (250, 253), (253, 257), (258, 262), (263, 270), (271, 272), (272, 273), (273, 274), (275, 278), (278, 280), (280, 283), (283, 287), (288, 294), (294, 295), (296, 298), (299, 303), (303, 305), (305, 309)

In [ ]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

Kan Kan


In [ ]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)


[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
answers = example["answers.text"]
start_char = example["answers.answer_start"]
end_char = start_char + len(answers)

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")


The answer is not in this feature.


In [ ]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers)

NameError: ignored

In [ ]:
pad_on_right = tokenizer.padding_side == "right"


In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers.text"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(examples["answers.answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = example["answers.answer_start"]
            end_char = start_char + len(answers)

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


In [ ]:
features = prepare_train_features(datasets['train'][:5])


In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Parameter 'function'=<function prepare_train_features at 0x7fb653975560> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

# Fine-tuning the model

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("deutsche-telekom/bert-multi-english-german-squad2")



Downloading:   0%|          | 0.00/676M [00:00<?, ?B/s]

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:


from transformers import default_data_collator

data_collator = default_data_collator

args = TrainingArguments(
    output_dir= "test_trainer",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,

)

In [ ]:
trainer = Trainer(
    model=model,
    args= args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
     data_collator=data_collator,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 133
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 27


Epoch,Training Loss,Validation Loss
1,No log,0.973304
2,No log,0.852150
3,No log,0.839062


***** Running Evaluation *****
  Num examples = 34
  Batch size = 16
***** Running Evaluation *****
  Num examples = 34
  Batch size = 16
***** Running Evaluation *****
  Num examples = 34
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=27, training_loss=1.1851384904649522, metrics={'train_runtime': 41.3835, 'train_samples_per_second': 9.642, 'train_steps_per_second': 0.652, 'total_flos': 104257405937664.0, 'train_loss': 1.1851384904649522, 'epoch': 3.0})

# Evaluation

In [ ]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [ ]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 512]), torch.Size([16, 512]))

In [ ]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'))

In [ ]:
n_best_size = 20

In [ ]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["Hash"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples


In [ ]:
validation_features = datasets["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["test"].column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 34
  Batch size = 16


In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))


In [ ]:
max_answer_length = 30


In [ ]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["test"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': -1.5839041,
  'text': 'Wählen Sie den Typ „FUNCTION“ aus dem angezeigten Dialog und beenden diesen mit OK'},
 {'score': -1.856517, 'text': 'beenden diesen mit OK'},
 {'score': -1.9158307, 'text': 'OK'},
 {'score': -1.9166288,
  'text': 'der Editor kann durch einen Klick auf Aktivieren geschlossen werden'},
 {'score': -1.971822,
  'text': 'durch einen Klick auf Aktivieren geschlossen werden'},
 {'score': -2.0108266, 'text': 'Klick auf Aktivieren geschlossen werden'},
 {'score': -2.058513, 'text': 'einen Klick auf Aktivieren geschlossen werden'},
 {'score': -2.1551905,
  'text': 'muss zuerst der ECU-TEST Tool-Server auf diesem Host installiert und gestartet werden'},
 {'score': -2.2523918, 'text': 'Führen Sie einen Rechtsklick'},
 {'score': -2.2691123,
  'text': 'Wählen Sie den Tab Plattform und erstellen Sie ein neues Funktionszugriff-Element'},
 {'score': -2.3149695,
  'text': 'Führen Sie einen Rechtsklick auf den „LABVIEW VI“-Eintrag'},
 {'score': -2.4639254,
  'text': 'Wäh

In [ ]:
datasets["test"][0]["answers.text"]


'Befindet sich LabVIEW auf einem Remote-Rechner, muss zuerst der ECU-TEST Tool-Server auf diesem Host installiert und gestartet werden. Anschließend können Sie für diesen Rechner einen neue Toolhost erstellen und finden „LABVIEW VI“ in der Toolliste dieses Eintrages vor.'

In [ ]:
import collections

examples = datasets["test"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["Hash"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["Hash"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if datasets:
            predictions[example["Hash"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["Hash"]] = answer

    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(datasets["test"], validation_features, raw_predictions.predictions)

Post-processing 20 example predictions split into 34 features.


  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
final_predictions

OrderedDict([('s8BXV/yCzrCWnpg26Nuqbg==',
              'Lesen“-Testschritt für diesen Parameter im OUT-Block ein'),
             ('FO8L0WYDI/fMJa1KapTSKw==', 'die SGBD-Jobs der Steuergeräte'),
             ('bVWU+hbl62lVTqkOZfjrBg==',
              'klicken Sie bei Filtergruppe auf den Neu-Button'),
             ('Ct6w4FM+aO8ovktEU9FfPw==',
              'ermöglicht die Konfiguration von Systemen'),
             ('iBaAAqyZ/cy+taMA8iHx7A==', 'Werte mit komplexem Aufbau'),
             ('3OYA7F2t1Or3kN6NrDdOxw==',
              'hat dies keinen Einfluss mehr auf die IfDef-Anweisung'),
             ('mhgvM+SZUp1TWWDTkBM9XQ==', 'Tab Package-Fehlerbehandlung'),
             ('rA+oy/dohYg4vXVfidB3rA==', 'Dauer'),
             ('4WjqSeJ/lRrUSNHNug71wQ==', 'Pin'),
             ('LT5k1v+3kYK6lLrBB2inoQ==',
              'I-Signal-I-PDU, Multiplexed-I-PDU, NM-PDU'),
             ('B5etfu8CQtl07YNyW9nbEQ==',
              'können neue Masken erstellt und bestehende Masken geöffnet werden'),
    

In [ ]:
metric = load_metric("accuracy")

In [ ]:
    formatted_predictions = [{"Hash": k, "prediction_text": v} for k, v in final_predictions.items()]
    references = [{"Hash": ex["Hash"], "answers.text": ex["answers.text"]} for ex in datasets["test"]]
formatted_predictions
references
#metric.compute(predictions=formatted_predictions, references=references)

NameError: ignored